# PSS

PSS(Parameter-Space Saliency)は，Deep Learningモデルの顕著性(Saliency)を可視化する手法の一つである．

誤分類に影響したパラメータ(Weight)を分析するアプローチで，影響の大きいパラメータを補正することでDeep Learningモデルの性能を改善できることが示された．

## 参考文献

* 論文
  * https://arxiv.org/abs/2108.01335
* GitHub
  * https://github.com/LevinRoman/parameter-space-saliency
* 解説資料：DL輪読会
  * https://www.slideshare.net/DeepLearningJP2016/dlwhere-do-models-go-wrong-parameterspace-saliency-maps-for-explainability

## PSS効果例

論文ではGrad-CAMとの比較が示されている．  
下図のようにPSSではGrad-CAMでは表現されない可視化要因を表現することができる(右2枚，ユキヒメドリ(junco)及び旅客列車(passenger car))．

![paper figure19](./figure/PSS/paper_figure19.png)

## PSS理論解説

本節では[PSS論文](https://arxiv.org/abs/2108.01335)及び[GitHub](https://github.com/LevinRoman/parameter-space-saliency)のソースコードをもとに解釈した内容を記載する．

論文では，パラメータ顕著性(Parameter saliency)の計算方法の説明(2.1 Parameter saliency profile)とモデルの誤動作を入力空間へ可視化する方法の説明(2.2 Input-space saliency for visualizing how filters malfunction)の2部構成で述べられる．

### Parameter saliency profile

パラメータ顕著性は下記の3ステップで計算する．

1. パラメータ毎の顕著性の計算
1. フィルタ毎の顕著性への集約
1. Validationデータによる標準化


#### パラメータ毎の顕著性の計算

入力$x$，正解ラベル$y$のValidationデータセット$D$，及び，損失関数$\mathcal{L}$で最小化したパラメータ$\theta$を持つ識別モデルを仮定する．

パラメータ毎の顕著性は，損失関数を識別モデルの各パラメータで偏微分して得られる勾配の大きさで定義する．  
インデックス$i$のパラメータを$\theta_i$で表すと，パラメータ毎の顕著性$s(x, y)_i$は以下のように定義される．

$$
  \begin{align}
    s(x, y)_i &:= |\nabla_{\theta_i}\mathcal{L}_\theta (x, y)|
  \end{align}
$$


#### フィルタ毎の顕著性への集約 

畳み込みフィルタはエッジ(Edge)，形状(Shape)，質感(Texture)を検出する性質があることで知られている．

顕著性$s(x, y)_i$をフィルタ毎に集約することにより，損失が最も敏感なフィルタを分離することが可能となる．つまり，分離されたフィルタを修正することによって，損失をより大きく減少させることが期待できる．

識別モデルの一つのフィルタを$\mathcal{F}_k$，フィルタ$\mathcal{F}_k$に属するパラメータのインデックス群を$\alpha_k$で示す．フィルタ毎の顕著性$\bar{s}(x, y)_k$は，パラメータ毎の顕著性をフィルタ単位で平均を求めるものとして，下記のように定義される．

$$
  \begin{align}
    \bar{s}(x, y)_k &:= \frac{1}{|\alpha_k|}\sum_{i \in \alpha_k}s(x, y)_i
  \end{align}
$$

ソースコードでは下記の通り，フィルタ毎の勾配としてカーネル毎に勾配の平均を算出する．

* [saliency_model_backprop.py](https://github.com/LevinRoman/parameter-space-saliency/blob/master/parameter_saliency/saliency_model_backprop.py#L49)
```python
for i in range(len(gradients)):  # Filter-wise aggregation
    # print(gradients[i].size())

    if self.aggregation == 'filter_wise':
        if len(gradients[i].size()) == 4:  # If conv layer
            if not self.signed:
                # first take abs and then aggregate
                filter_grads.append(gradients[i].abs().mean(-1).mean(-1).mean(-1))
            else:
                filter_grads.append(gradients[i].mean(-1).mean(-1).mean(-1))
    if self.aggregation == 'parameter_wise':
        if not self.signed:
            filter_grads.append(gradients[i].view(-1).abs())
        else:
            filter_grads.append(gradients[i].view(-1))
    if self.aggregation == 'tensor_wise':
        raise NotImplementedError
```

#### Validationデータによる標準化

下図(論文Figure1)の上図は，ResNet-50で層毎の顕著性を，ImageNetのValidationデータセットに対して平均値を算出し，層毎に顕著性降順にソートしたグラフである．

![paper figure1](./figure/PSS/paper_figure1.png)

勾配のスケールが入力層から出力層の間で異なっていることが明らかである(入力層の顕著性が大きく，出力層に向かうにつれて小さくなる)．これにはいくつかの要因がある．

1. 入力層に近いフィルタは，エッジ(Edge)や質感(Texture)等，幅広い画像に対して有効な特徴量を抽出する性質を持つ．
  * つまり，タスクに特化したフィルタではない為，出力層のフィルタと比較した際に相対的に損失が大きくなる
1. 一般的にネットワークを構成する際は入力層に近いほどフィルタ数が少なくなるように設計する．層あたりのフィルタ数が少ないと，各フィルタが及ぼす影響力が相対的に大きくなる．
1. 入力層に近いフィルタの効果は，後続のネットワークへ継承される．
  * つまり，出力層に向かうにつれて入力層側のフィルタで獲得した特徴量を破壊しないように影響度が小さくなる
  
そこで，スケールをフィルタ間で合わせるために，フィルタ毎にValidationデータセットで標準化する．フィルタ$k$の標準化顕著性$\hat{s}(x, y)_k$は下記のように定義される．

$$
  \begin{align}
    \hat{s}(x, y)_k &:= \frac{|\bar{s}(x, y)_k - \mu_k|}{\sigma_k}
  \end{align}
$$

これを一般化すると，

$$
  \begin{align}
    \hat{s}(x, y) &:= \frac{|\bar{s}(x, y) - \mu|}{\sigma}
  \end{align}
$$

となり，畳み込みフィルタ数長のテンソルが$\hat{s}(x, y)$として得られる．


### Input-space saliency for visualizing how filters malfunction

上述の方法で算出した顕著性を用いて，ネットワークの誤動作や異常動作の要因となるフィルタを特定することが可能となる．

具体的には，大別して下記の3ステップにより，フィルタの顕著性に影響する画像特徴量を特定することができる．

1. 上位$k$個のフィルタ顕著性を選択する  
※$k$は任意で，ソースコードでは引数で個数を指定
1. 選択したフィルタ顕著性を定数倍(Boost)して$s'$を算出する
1. Boost前後の顕著性($s, s'$)のコサイン類似度を計算し，その勾配の絶対値を算出する($M_F$)  
$$
  \begin{align}
    M_F = |\nabla_x D_C(s(x, y), s')|
  \end{align}
$$

算出された$M_F$がフィルタ$F$の顕著性に影響を与えるピクセルの影響度合いを示す．


* [parameter_and_input_saliency.py](https://github.com/LevinRoman/parameter-space-saliency/blob/master/parameter_and_input_saliency.py#L132)

```python
#Errors are a fragile concept, we should not perturb too much, we will end up on the object
for noise_iter in range(args.noise_iters):
    perturbed_inputs = reference_inputs.detach().clone()
    perturbed_inputs = (1-args.noise_percent)*perturbed_inputs + args.noise_percent*torch.randn_like(perturbed_inputs)

    perturbed_outputs = net(perturbed_inputs)
    _, perturbed_predicted = perturbed_outputs.max(1)
    # print(readable_labels[int(perturbed_predicted[0])])

    #Backprop to the input
    perturbed_inputs.requires_grad_()
    #Find the true saliency:
    filter_saliency = filter_saliency_model(
        perturbed_inputs, reference_targets,
        testset_mean_abs_grad=testset_mean_stat,
        testset_std_abs_grad=testset_std_stat).to(device)

    #Find the top-k salient filters
    if args.compare_random:
        sorted_filters = torch.randperm(filter_saliency.size(0)).cpu().numpy()
    else:
        sorted_filters = torch.argsort(filter_saliency, descending=True).cpu().numpy()

    #Boost them:
    filter_saliency_boosted = filter_saliency.detach().clone()
    filter_saliency_boosted[sorted_filters[:args.k_salient]] *= args.boost_factor

    #Form matching loss and take the gradient:
    matching_criterion = torch.nn.CosineSimilarity()
    matching_loss = matching_criterion(filter_saliency[None, :], filter_saliency_boosted[None, :])
    matching_loss.backward()

    grads_to_save = perturbed_inputs.grad.detach().cpu()
    grad_samples.append(grads_to_save)
#Find averaged gradients (smoothgrad-like)
grads_to_save = torch.stack(grad_samples).mean(0)
```


※下記でヒートマップを生成しているが，コサイン類似度の勾配の絶対値が意味するものが不明

* [parameter_and_input_saliency.py](https://github.com/LevinRoman/parameter-space-saliency/blob/0e3b3d69c6e222aee6af0264d7ce3ddc6d19744e/parameter_and_input_saliency.py#L88)

```python
    grads_to_save = (grads_to_save - np.min(grads_to_save)) / (np.max(grads_to_save) - np.min(grads_to_save))

    #Superimpose gradient heatmap
    reference_image_to_compare = inv_transform_test(reference_image[0].cpu()).permute(1, 2, 0)
    gradients_heatmap = np.ones_like(grads_to_save) - grads_to_save
    gradients_heatmap = cv2.GaussianBlur(gradients_heatmap, (3, 3), 0)

    #Save the heatmap
    heatmap_superimposed = show_heatmap_on_image(reference_image_to_compare.detach().cpu().numpy(), gradients_heatmap)
    plt.imshow(heatmap_superimposed)
    plt.axis('off')
    plt.savefig(os.path.join(save_path, 'input_saliency_heatmap_{}.png'.format(save_name)), bbox_inches='tight')
    print('Input space saliency saved to {} \n'.format(os.path.join(save_path, 'input_saliency_heatmap_{}.png'.format(save_name))))
```

## PSS動作確認

In [1]:
import os

In [2]:
if (not os.path.exists("parameter-space-saliency")):
    !git clone https://github.com/LevinRoman/parameter-space-saliency
    !cd parameter-space-saliency ; git checkout 0e3b3d69c6e222aee6af0264d7ce3ddc6d19744e

Cloning into 'parameter-space-saliency'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 143 (delta 61), reused 92 (delta 24), pack-reused 0
Receiving objects: 100% (143/143), 4.68 MiB | 6.71 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Note: checking out '0e3b3d69c6e222aee6af0264d7ce3ddc6d19744e'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 0e3b3d6 fixing filter saliency


In [3]:
!cd parameter-space-saliency ; pip install -r requirements.txt

     |████████████████████████████████| 97 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 50.4 MB 25.7 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 93.5 MB/s eta 0:00:01
     |████████████████████████████████| 640 kB 79.6 MB/s eta 0:00:01
     |████████████████████████████████| 22.2 MB 95.3 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 29.2 MB/s eta 0:00:01
     |████████████████████████████████| 292 kB 96.2 MB/s eta 0:00:01
     |████████████████████████████████| 776.8 MB 32 kB/s s eta 0:00:01     |████████▊                       | 213.1 MB 19.9 MB/s eta 0:00:29
     |████████████████████████████████| 12.8 MB 26.3 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 3.0 MB/s  eta 0:00:01
     |████████████████████████████████| 137 kB 87.7 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 72.5 MB/s eta 0:00:01
     |████████████████████████████████| 113 kB 90.1 MB/s eta 0:00:01
     |███████████████████████

In [4]:
!cd parameter-space-saliency ; python parameter_and_input_saliency.py --model resnet50 --image_path raw_images/great_white_shark_mispred_as_killer_whale.jpeg --image_target_label 2

==> Preparing data..

               ImageNet validation set path is not specified.
               The code will only work with raw --image_path and --image_target_label specified.
               In this scenario, --reference_id must be None.
              
parameter_and_input_saliency.py:241: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  readable_labels = yaml.load(readable_labels)
==> Building model..
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████████████████████████████████| 97.8M/97.8M [00:01<00:00, 97.1MB/s]
Total filters: 26560
Total layers: 53


        Using image raw_images/great_white_shark_mispred_as_killer_whale.jpeg
        and target label 2

        


        Image target label: 2
        Image target class name: great white shark, white shark, man-eater

In [5]:
!ls parameter-space-saliency/figures

filter_saliency_107_densenet121.png
filter_saliency_107_inception_v3.png
filter_saliency_107_resnet50.png
filter_saliency_107_vgg19.png
filter_saliency_great_white_shark_mispred_as_killer_whale_resnet50.png
input_space_saliency


In [6]:
!ls parameter-space-saliency/figures/input_space_saliency

input_saliency_heatmap_107_densenet121.png
input_saliency_heatmap_107_inception_v3.png
input_saliency_heatmap_107_resnet50.png
input_saliency_heatmap_107_vgg19.png
input_saliency_heatmap_great_white_shark_mispred_as_killer_whale_resnet50.png


### 実行結果

#### DenseNet 121

##### Filter Saliency

![DenseNet Filter Saliency](parameter-space-saliency/figures/filter_saliency_107_densenet121.png)

##### Input Saliency Heatmap

![DenseNet Heatmap](parameter-space-saliency/figures/input_space_saliency/input_saliency_heatmap_107_densenet121.png)

#### Inception v3

##### Filter Saliency

![Inception v3 Filter Saliency](parameter-space-saliency/figures/filter_saliency_107_inception_v3.png)

##### Input Saliency Heatmap

![Inception V3 Heatmap](parameter-space-saliency/figures/input_space_saliency/input_saliency_heatmap_107_inception_v3.png)

#### ResNet50

##### Filter Saliency

![ResNet50 Filter Saliency](parameter-space-saliency/figures/filter_saliency_107_resnet50.png)

##### Input Saliency Heatmap

![ResNet50 Heatmap](parameter-space-saliency/figures/input_space_saliency/input_saliency_heatmap_107_resnet50.png)

#### ResNet50

##### Filter Saliency

![VGG19 Filter Saliency](parameter-space-saliency/figures/filter_saliency_107_vgg19.png)

##### Input Saliency Heatmap

![VGG19 Heatmap](parameter-space-saliency/figures/input_space_saliency/input_saliency_heatmap_107_vgg19.png)